# 03-Movie-Query

![](https://images.unsplash.com/photo-1521967906867-14ec9d64bee8?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1050&q=80)

Photo by [JESHOOTS.COM](https://unsplash.com/photos/PpYOQgsZDM4)

In this exercise, you will make a prototype of movie recommender based on a query.

Basically, we want to type a word or a text, and to find the closest movies to this text query. In order to do so, we will use movie title but also movie overview.

Begin by importing usual libraries:

In [90]:
# TODO: import needed libraries
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Import the data into the file *movies_overview.csv*

In [91]:
# TODO: Load the dataset movies_overview.csv
df = pd.read_csv('movies_overviews.csv',on_bad_lines='skip')


Check what is in this file and how is the data.

In [92]:
# TODO: Check the dataset
df.info()

## no null values


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013 entries, 0 to 1012
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   original_title  1013 non-null   object
 1   overview        1013 non-null   object
dtypes: object(2)
memory usage: 16.0+ KB


In [93]:
df.columns

Index(['original_title', 'overview'], dtype='object')

Concatenate the title and overview into another column.

In [94]:
# TODO: Concatenate title and overview
df['title_overview'] = df['original_title'].apply(lambda x: x + ': ') + df['overview']
df['title_overview']

0       Minions: Minions Stuart, Kevin and Bob are rec...
1       Wonder Woman: An Amazon princess comes to the ...
2       Beauty and the Beast: A live-action adaptation...
3       Baby Driver: After being coerced into working ...
4       Big Hero 6: The special bond that develops bet...
                              ...                        
1008    LOL: In a world connected by YouTube, iTunes, ...
1009    God Bless America: Fed up with the cruelty and...
1010    The Dead Lands: Hongi, a Maori chieftain’s tee...
1011    Scream 4: Sidney Prescott, now the author of a...
1012    Nine Lives: A stuffy businessman finds himself...
Name: title_overview, Length: 1013, dtype: object

Compute both the BOW and the TF-IDF of this new column using scikit learn.

In [95]:
def preprocessing(document):
    # 1- tokenization
    tokens = word_tokenize(document) ## ok avec skip dans read_csv
    # 2- punctuation removal
    tokens = [t.lower() for t in tokens if t.isalpha()]
    # 3- remove stopwords
    stop_words = stopwords.words('english')
    tokens = [t for t in tokens if not t in stop_words]
    # 4- stemming
    stemmer = PorterStemmer() #build root by removing some known suffix and prefix
    tokens_stem = [stemmer.stem(w) for w in tokens]
    return tokens_stem

In [96]:
# TODO: compute the BOW and TF-IDF ON THE NEW COL: title_overview

### BOW
df['title_overview'] = df['title_overview'].apply(preprocessing)

BOW_vectorizer = CountVectorizer(analyzer=lambda x: x)
#BOW = vectorizer.fit_transform([preprocessing(x) for x in df.headline_text]).toarray()
BOW = BOW_vectorizer.fit_transform(df.title_overview).toarray()
print(f'BOW.shape: {BOW.shape}') #1013x6536
strings = BOW_vectorizer.get_feature_names_out()

df_BOW = pd.DataFrame(data=BOW, columns=strings)  ## être sûr du phasage entre les valeurs et les colonnes
df_BOW #1013x6536


BOW.shape: (1013, 6356)


,aadland,aaron,abandon,abbey,abbi,abdic,abduct,abel,abernathi,abi,...,zeu,zhong,zimmer,zombi,zone,zoo,zookeep,zooland,zootopia,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
## TF-IDF: le preprocessing a déjà été appliqué à la colonne title_overview
TFIDF_vectorizer = TfidfVectorizer(analyzer=lambda x: x)
tf_idf = TFIDF_vectorizer.fit_transform(df.title_overview).toarray()
tf_idf

df_TFIDF = pd.DataFrame(data=tf_idf, columns=TFIDF_vectorizer.get_feature_names_out())
df_TFIDF

,aadland,aaron,abandon,abbey,abbi,abdic,abduct,abel,abernathi,abi,...,zeu,zhong,zimmer,zombi,zone,zoo,zookeep,zooland,zootopia,zuckerberg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


What are the dimensions of the TF-IDF and the BOW? Print them out and explain them.

In [98]:
# TODO: print the shapes of TF-IDF and BOW
print(f'df BOW shape: {df_BOW.shape}')
print(f'df TI-IDF shape: {df_TFIDF.shape}')


df BOW shape: (1013, 6356)
df TI-IDF shape: (1013, 6356)


Now we want to define a function that takes as input a text query and prints the 10 most similar movies according to the BOW.

To do that, the function should make the following steps:
* Compute the BOW of the input text query
* Compute cosine similarity between the query and the movies
* Print the 10 most similar movies

The function will have the following signature:

`get_BOW_similar(query, movies_df, BOW, BOW_vectorizer)`

In [99]:
# TODO: implement the function get_BOW_similar (on fera à la suite sur la base de TF-IDF)
"""
- input:
takes as input a text query
- return:
prints the 10 most similar movies according to the BOW
"""
def get_BOW_similar(query, movies_df, BOW, BOW_vectorizer):
    # BOW of the input text query
    ###df_query_BOW = pd.DataFrame(BOW_vectorizer.transform(query).toarray(), columns=BOW_vectorizer.get_feature_names_out())
    query_BOW = BOW_vectorizer.transform(query).toarray()

    # cosine similarity between the query and the movies
    similarity = cosine_similarity(query_BOW, BOW)
    print(f"Similarity.max(): {similarity.max()}")
    print(f'similarity.shape: {similarity.shape}')

    # Return the closest movie
    #movies_df[similarity.argmax()]

    ###print(f'10 most similar movies: {query_BOW.max().sort_values(ascending=False)[:10]}')
    print(movies_df[similarity.argsort()[0][-10:][::-1]])  ## correctif

    ##most_similar_movies = [movies_df.loc[i] for i in similarity.sort()[-10:].index]
    ##most_similar_movies
    return similarity


Now try to use that function with several queries, check how it works:

In [100]:
# TODO: test yout get_BOW_similar: ça marche indifféremment avec BOW ou df_BOW
query = ['zerzer']
movies_df = df.original_title
similarity = get_BOW_similar(query, movies_df, BOW, BOW_vectorizer)


Similarity.max(): 0.1543033499620919
similarity.shape: (1, 1013)
144              The Lost City of Z
101                     World War Z
1012                     Nine Lives
347                            Noah
345                      Blood Ties
344        Guns, Girls and Gambling
343                           Filth
342                           Nerve
341     The Diary of a Teenage Girl
340             You Don't Know Jack
Name: original_title, dtype: object


Now let's do the same function with TF-IDF to see if it works better:

`get_TFIDF_similar(query, movies_df, TFIDF, TFIDF_vectorizer)`

Then test it on some queries.

In [103]:
# TODO: implement the function get_TFIDF_similar
def get_TFIDF_similar(query, movies_df, TFIDF, TFIDF_vectorizer):
    query_TFIDF = TFIDF_vectorizer.transform(query).toarray()

    # cosine similarity between the query and the movies
    similarity = cosine_similarity(query_TFIDF, TFIDF)
    print(f"Similarity.max(): {similarity.max()}")
    print(f'similarity.shape: {similarity.shape}')
    print(movies_df[similarity.argsort()[0][-10:][::-1]])  ## correctif
    return similarity

In [104]:
# TODO: test get_TDIDF_similar
query = ['zerzer']
movies_df = df.original_title
similarity = get_TFIDF_similar(query, movies_df, df_TFIDF, BOW_vectorizer)
## meme résultat qu'avec BOW sur cet exemple

Similarity.max(): 0.18094997973075308
similarity.shape: (1, 1013)
144              The Lost City of Z
101                     World War Z
1012                     Nine Lives
347                            Noah
345                      Blood Ties
344        Guns, Girls and Gambling
343                           Filth
342                           Nerve
341     The Diary of a Teenage Girl
340             You Don't Know Jack
Name: original_title, dtype: object


What can you do now to improve your function?

If you have time, you can add preprocessing on your query, and before the TF-IDF/BOW, this would probably improve the performances.